# 🎓 GenZ Mentor – A GenAI Career Guide for Students from Underserved Backgrounds

In this project, I've built a **kind, AI-powered mentor** that gives personalized career advice to students—especially those who don’t have access to real mentors or career resources. 

This GenAI mentor listens, understands, and replies with real career paths, helpful resources, and *a splash of motivation* 🌈

---

### 🌟 What this notebook does:
- Understands a student's interests/goals
- Uses GenAI to offer career suggestions + learning tips
- Gives wholesome encouragement




In [1]:
!pip install -q google-generativeai faiss-cpu sentence-transformers pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm

In [2]:
import os
import warnings

# Suppress TensorFlow and CUDA related warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # TensorFlow log level suppression
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow')  # General TensorFlow warnings suppression
warnings.filterwarnings("ignore", category=UserWarning, module='torch')  # PyTorch warnings suppression


In [3]:
import google.generativeai as genai
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import os


E0000 00:00:1745087091.392860      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745087091.460142      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")


## 📂 Step 1: Load the Student Career Dataset

We’re using a public Kaggle dataset that contains **student interests and career goals**.  
This will help our GenZ Mentor recommend real career paths that *match* the user’s background.


In [5]:
# Load the dataset
df = pd.read_csv("/kaggle/input/understanding-career-aspirations-of-genz/Your Career Aspirations of GenZ.csv")

# Display the first few rows
df.head()


,Your Current Country.,Your Current Zip Code / Pin Code,Your Gender,Which of the below factors influence the most about your career aspirations ?,Would you definitely pursue a Higher Education / Post Graduation outside of India ? If only you have to self sponsor it.,How likely is that you will work for one employer for 3 years or more ?,Would you work for a company whose mission is not clearly defined and publicly posted.,How likely would you work for a company whose mission is misaligned with their public actions or even their product ?,How likely would you work for a company whose mission is not bringing social impact ?,What is the most preferred working environment for you.,Which of the below Employers would you work with.,Which type of learning environment that you are most likely to work in ?,Which of the below careers looks close to your Aspirational job ?,What type of Manager would you work without looking into your watch ?,Which of the following setup you would like to work ?
0,India,273005,Male,People who have changed the world for better,"Yes, I will earn and do that","This will be hard to do, but if it is the righ...",No,Will NOT work for them,4,Fully Remote with No option to visit offices,Employer who rewards learning and enables that...,"Instructor or Expert Learning Programs, Trial ...","Business Operations in any organization, Build...","Manager who explains what is expected, sets a ...","Work alone, Work with 2 to 3 people in my team..."
1,India,851129,Male,People who have changed the world for better,"No, But if someone could bare the cost I will","This will be hard to do, but if it is the righ...",No,Will NOT work for them,1,Fully Remote with Options to travel as and whe...,Employer who pushes your limits by enabling an...,"Self Paced Learning Portals, Instructor or Exp...","Business Operations in any organization, Build...","Manager who explains what is expected, sets a ...",Work with 5 to 6 people in my team
2,India,123106,Female,Social Media like LinkedIn,"Yes, I will earn and do that",Will work for 3 years or more,Yes,Will work for them,7,Hybrid Working Environment with less than 15 d...,Employer who pushes your limits by enabling an...,"Self Paced Learning Portals, Trial and error b...",Manage and drive End-to-End Projects or Produc...,"Manager who explains what is expected, sets a ...","Work with 2 to 3 people in my team, Work with ..."
3,India,834003,Male,"People from my circle, but not family members","No, But if someone could bare the cost I will","This will be hard to do, but if it is the righ...",No,Will NOT work for them,6,Hybrid Working Environment with less than 15 d...,Employer who pushes your limits by enabling an...,"Instructor or Expert Learning Programs, Trial ...","Business Operations in any organization, Manag...","Manager who explains what is expected, sets a ...",Work with 2 to 3 people in my team
4,India,301019,Female,Influencers who had successful careers,"No, But if someone could bare the cost I will",Will work for 3 years or more,No,Will NOT work for them,5,Fully Remote with Options to travel as and whe...,Employer who appreciates learning and enables ...,"Self Paced Learning Portals, Learning by obser...",Teaching in any of the institutes/online or Of...,"Manager who explains what is expected, sets a ...","Work with 2 to 3 people in my team, Work with ..."


## ✨ Step 2: Embed the Career Info

We’ll turn each student profile into a vector using a Sentence Transformer.  
That way, when someone types in their background, we can find the most *similar* student in our dataset!


In [6]:
df.columns


Index(['Your Current Country.', 'Your Current Zip Code / Pin Code',
       'Your Gender',
       'Which of the below factors influence the most about your career aspirations ?',
       'Would you definitely pursue a Higher Education / Post Graduation outside of India ? If only you have to self sponsor it.',
       'How likely is that you will work for one employer for 3 years or more ?',
       'Would you work for a company whose mission is not clearly defined and publicly posted.',
       'How likely would you work for a company whose mission is misaligned with their public actions or even their product ?',
       'How likely would you work for a company whose mission is not bringing social impact ?',
       'What is the most preferred working environment for you.',
       'Which of the below Employers would you work with.',
       'Which type of learning environment that you are most likely to work in ?',
       'Which of the below careers looks close to your Aspirational job ?',
   

In [7]:
df['combined'] = (
    "Career Aspiration: " + df['Which of the below careers looks close to your Aspirational job ?'] + 
    " | Influencing Factor: " + df['Which of the below factors influence the most about your career aspirations ?'] +
    " | Preferred Environment: " + df['What is the most preferred working environment for you.']
)


In [8]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for the 'combined' column
embeddings = model.encode(df['combined'].tolist())


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
import numpy as np

# Add the embeddings to your dataframe (each embedding is a list of numbers)
df['embeddings'] = list(embeddings)


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def find_best_match(query):
    query_embedding = model.encode([query])  # Get the query embedding
    similarities = cosine_similarity(query_embedding, embeddings)  # Compare with the embeddings
    best_match_index = np.argmax(similarities)  # Find the index of the most similar response
    return df.iloc[best_match_index]['combined']  # Return the matched career aspiration text


In [11]:
# Uncomment this cell to test with custom prompts
# while True:
#    query = input("Ask a career-related question: ")
#    if query.lower() == 'exit':
#        print("Goodbye!")
#        break
#    response = find_best_match(query)
#    print(f"AI's response: {response}")
#    print("="*50)
